# Speckle Tracking

In [1]:
from pathlib import Path
# from project_heart.lv import LV
from project_heart.utils import set_jupyter_backend
from project_heart.enums import *
set_jupyter_backend("pythreejs")
import logging
import numpy as np
float_formatter = "{:.5f}".format
np.set_printoptions(formatter={'float_kind':float_formatter})

from project_heart.examples import get_lv_ideal
from project_heart.examples import get_lv_typeA

lv_ideal = get_lv_ideal(Path("../../_static/sample_files/ideal_linear_pressure_increase copy.xplt"))
lv_typeA = get_lv_typeA(
    Path("../../_static/sample_files/lv_typeA_hex.vtk"),
    Path("../../_static/sample_files/sample_displacement_lv_typeA_hex_states_linear_press_incr.pbz2"),
    save_spk_dict=False
    )

sample_spk_typeA_endo = lv_typeA.get_speckles(spk_name="SAMPLE", spk_group="endo", spk_collection="SAMPLE")[0]
sample_spk_typeA_epi = lv_typeA.get_speckles(spk_name="SAMPLE", spk_group="epi", spk_collection="SAMPLE")[0]


import ipywidgets as widgets
from ipywidgets import interactive
from ipywidgets import Button, HBox, VBox
from IPython.display import display, clear_output

plot_clusters_wdg = widgets.Dropdown(options=["subsets", "clusters"], description="categories")

mode_wdg = widgets.Dropdown(options=["longitudinal", "transversal"], description="mode")
nodeset_wdg = widgets.Dropdown(options=["endo", "epi"], description="nodeset")

d_wdg = widgets.FloatSlider(value=2.5, min=0.0, max=8.0, description="d", continuous_update=False)
k_wdg = widgets.FloatSlider(value=0.5, min=0.0, max=1.0, step=0.05, description="k", continuous_update=False)

k_min_wdg = widgets.FloatSlider(value=-1, min=-1.0, max=1.0, step=0.05, description="k_min", continuous_update=False)
k_max_wdg = widgets.FloatSlider(value=-1, min=-1.0, max=1.0, step=0.05, description="k_max", continuous_update=False)

normal_a_wdg = widgets.FloatSlider(value=0.0, min=0.0, max=180.0, description="a", continuous_update=False)

n_subsets_wdg = widgets.IntSlider(value=6, min=0, max=15, description="n_subsets", continuous_update=False)
subset_criteria_wdg = widgets.Dropdown(value="z2", options=["z", "z2", 'angles', 'angles2', 'angles3'], description="subset_criteria")

n_clusters_wdg = widgets.IntSlider(value=6, min=0, max=60, description="n_clusters", continuous_update=False)
cluster_criteria_wdg = widgets.Dropdown(value="angles3", options=["z", "z2", 'angles', 'angles2', 'angles3'], description="cluster_criteria")

a = VBox([plot_clusters_wdg, mode_wdg, nodeset_wdg])
b = VBox([d_wdg, k_wdg, normal_a_wdg])
c = VBox([n_subsets_wdg, subset_criteria_wdg, k_min_wdg])
d = VBox([n_clusters_wdg, cluster_criteria_wdg, k_max_wdg])

controls = HBox([a, b, c, d])

def create_and_show_speckles(plot_mode, mode, nodeset, normal_a, d, k, 
                             n_subsets, subset_criteria,
                             n_clusters, cluster_criteria,
                             k_min, k_max
                             ):
    # clear_output()
    
    if mode == "longitudinal":
        a = np.radians(normal_a)
        normal = [np.cos(a),np.sin(a),0.0]
    else:
        normal = [0.,0.,1.]
    
    if nodeset == "endo":
        nodeset = LV_SURFS.ENDO
    else:
        nodeset = LV_SURFS.EPI
        
    # plot spkes on ideal case
    spk_name = "SAMPLE-INTERACT-" + str(nodeset) 
    spk_name = "-" + str(normal)
    spk_name += "-" + str(n_subsets) + str(subset_criteria)
    spk_name += "-" +str(n_clusters) + str(cluster_criteria)
    
    spk1 = lv_ideal.create_speckles(
                collection=spk_name,
                group=spk_name,
                name=spk_name,
                from_nodeset=nodeset,
                # exclude_nodeset=[LV_SURFS.BASE, LV_SURFS.ENDO],
                d=d,
                k=k,
                normal_to=normal,
                n_subsets=n_subsets,
                subsets_criteria=subset_criteria,
                n_clusters=n_clusters,
                cluster_criteria=cluster_criteria,
                t=0.0,
                kmin=k_min,
                kmax=k_max,
                log_level=logging.WARN,
            )
    
    if plot_mode == "subsets":
        lv_ideal.plot_speckles(spk1, 
                            cmap="tab20",
                            point_size=300,
                            add_k_centers=True, 
                            k_centers_as_line=False, 
                            k_bins=False,
                            plot_kwargs=dict(notebook=True)
                            )
    else:
        lv_ideal.plot_speckles(spk1, 
                            cmap="hot",
                            point_size=300,
                            add_k_centers=True, 
                            k_centers_as_line=True, 
                            k_bins=True,
                            plot_kwargs=dict(notebook=True)
                            )
    
    # plot speckles on non-ideal case.
    spk2 = lv_typeA.create_speckles(
                collection=spk_name,
                group=spk_name,
                name=spk_name,
                from_nodeset=nodeset,
                # exclude_nodeset=[LV_SURFS.BASE, LV_SURFS.ENDO],
                d=d,
                k=k,
                normal_to=normal,
                n_subsets=n_subsets,
                subsets_criteria=subset_criteria,
                n_clusters=n_clusters,
                cluster_criteria=cluster_criteria,
                t=0.0,
                kmin=k_min,
                kmax=k_max,
                log_level=logging.WARN,
            )
    
    if plot_mode == "subsets":
        lv_typeA.plot_speckles(spk2, 
                            cmap="tab20",
                            point_size=300,
                            add_k_centers=True, 
                            k_centers_as_line=False, 
                            k_bins=False,
                            plot_kwargs=dict(notebook=True)
                            )
    else:
        lv_typeA.plot_speckles(spk2, 
                            cmap="hot",
                            point_size=300,
                            add_k_centers=True, 
                            k_centers_as_line=True, 
                            k_bins=True,
                            plot_kwargs=dict(notebook=True)
                            )
    
def interactive_speckles():
    out = widgets.interactive_output(create_and_show_speckles, 
                                 dict(
                                     plot_mode=plot_clusters_wdg,
                                    mode=mode_wdg, 
                                    nodeset=nodeset_wdg, 
                                    normal_a=normal_a_wdg,
                                    d=d_wdg, k=k_wdg, 
                                    n_subsets=n_subsets_wdg, 
                                    subset_criteria=subset_criteria_wdg,
                                    n_clusters=n_clusters_wdg, 
                                    cluster_criteria=cluster_criteria_wdg,
                                    k_min=k_min_wdg,
                                    k_max=k_max_wdg,
                                  ))
    display(controls, out)

## What are Speckles?

DOC PENDING...

In [2]:
interactive_speckles()

Output()

## Sample Speckles

### Longitudinal Speckles

In [3]:
# lv_ideal.plot_speckles(endo_long[1], add_centers=True, cmap="tab10", categories=True)

In [4]:
# lv_ideal.plot_speckles(epi_long[0], cmap="jet", add_k_centers=True, k_centers_as_line=True, k_bins=True, categories=True)

### Circumferential Speckles

In [5]:
# lv_ideal.plot_speckles(endo_circ[7], add_centers=True, cmap="tab10", categories=True)

In [6]:
# lv_ideal.plot_speckles(epi_circ[7], cmap="jet", add_k_centers=True, k_centers_as_line=True, k_bins=True, categories=True)